## SQLITE

**1**: Read all the data from bc.csv, how many rows is there ? Columns ? 

**2**: Insert the bc.csv in the sqlite format.

**3**: In the same db create a new table called tomatch with two variables id_ and rdm_float. 

**4**: In the table tomatch insert observations where the id_ starts from 50k, ends at 10000000 and is only even numbers. rdm_float is a random float for each observation.

**5**: Do an Inner join between the two tables and compute the different aggregate functions on a column of your choice.

**6**: How many observations have a radius_mean greater than 15 ? From them how many have a texture_mean greater than 20 ?

**7**: For each category in diagnosis do the mean of a variable and round it to get only 2 numbers after the comma. Compute the difference between the mean of both groups


**8**: Change the name of the column "diagnosis" to "label"

**9**: Add a column named "day" of type DATE.

**10**: Change the type of the column "area_mean" to int (hint, not possible on sqlite, create a new column)

**11**: create a new table called tomatch2 exactly the same way as tomatch except you increment id_ by 10.

**12**: Do an inner join of bc, tomatch and tomatch2 where the radius_mean is greater than 15, and the rdm_float is greater than 0.50 in tomatch AND tomatch2. 


In [ ]:
import sqlite3
import pandas as pd
import numpy as np

#SQLite dataase connection
conn = sqlite3.connect('bc_data.db')
cursor = conn.cursor()

In [ ]:
# **1**: Read all the data from bc.csv, how many rows and columns
df = pd.read_csv('data/Chap1/bc.csv')
print(f'Rows: {df.shape[0]}, Columns: {df.shape[1]}')

In [ ]:
# **2**: Insert the bc.csv into SQLite
df.to_sql('bc', conn, if_exists='replace', index=False)

In [ ]:
# **3**: Create a new table called tomatch
cursor.execute('''
CREATE TABLE tomatch (
    id_ INTEGER,
    rdm_float REAL
)
''')

In [ ]:
# **4**: Insert observations in tomatch
cursor.executemany('''
INSERT INTO tomatch (id_, rdm_float) VALUES (?, ?)
''', [(i, np.random.rand()) for i in range(50000, 10000001, 2)])

In [ ]:
# **5**: Inner join between the two tables and aggregate functions
result = pd.read_sql_query('''
SELECT AVG(bc.radius_mean) AS avg_radius
FROM bc
JOIN tomatch ON bc.id = tomatch.id_
''', conn)
print(result)

In [ ]:
# **6**: Observations with radius_mean > 15 and texture_mean > 20
count_radius = pd.read_sql_query('''
SELECT COUNT(*) FROM bc WHERE radius_mean > 15
''', conn)
count_texture = pd.read_sql_query('''
SELECT COUNT(*) FROM bc WHERE radius_mean > 15 AND texture_mean > 20
''', conn)
print(count_radius, count_texture)

In [ ]:
# **7**: Mean of a variable for each category in diagnosis
mean_diff = pd.read_sql_query('''
SELECT diagnosis, ROUND(AVG(radius_mean), 2) AS rounded_mean
FROM bc
GROUP BY diagnosis
''', conn)
print(mean_diff)

In [ ]:
# **8**: Change the name of the column "diagnosis" to "label"
cursor.execute('''
ALTER TABLE bc RENAME COLUMN diagnosis TO label
''')

In [ ]:
# **9**: Add a column "day" of type DATE
cursor.execute('''
ALTER TABLE bc ADD COLUMN day DATE
''')

In [ ]:
# **10**: Change the type of "area_mean" to int by creating a new column
cursor.execute('''
ALTER TABLE bc ADD COLUMN area_mean_int INTEGER
''')
cursor.execute('''
UPDATE bc SET area_mean_int = CAST(area_mean AS INTEGER)
''')

In [ ]:
# **11**: Create new table tomatch2 similar to tomatch but increment id_ by 10
cursor.execute('''
CREATE TABLE tomatch2 AS SELECT id_ + 10 AS id_, rdm_float FROM tomatch
''')

In [ ]:
# **12**: Inner join of bc, tomatch, and tomatch2 with conditions
final_result = pd.read_sql_query('''
SELECT *
FROM bc
JOIN tomatch ON bc.id = tomatch.id_ AND tomatch.rdm_float > 0.50
JOIN tomatch2 ON bc.id = tomatch2.id_ AND tomatch2.rdm_float > 0.50
WHERE bc.radius_mean > 15
''', conn)
print(final_result)

In [ ]:
# Commit changes and close connection
conn.commit()
conn.close()

## MySQL

**1**: How many orders were placed in total?

**2**: What is the total sales amount?

**3**: What is the average profit per order?

**4**: Which customer has placed the most orders?

**5**: What is the total profit for each product category?

**6**: What is the average discount percentage?

**7**: Which region has the highest total sales?

**8**: How many orders were shipped using each shipping mode?

**9**: What is the total quantity ordered for each product sub-category?

**10**: Which product has the highest profit margin?

**11**: What is the total sales amount for each year?

**12**: Which customer segment has the highest average sales?


In [ ]:
import mysql.connector
import pandas as pd 

# Connect to the MySQL database
db = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    passwd="root",
)
cursor = db.cursor()
cursor.execute("CREATE DATABASE bc_data2")

df = pd.read_csv('data/Chap1/bc.csv')

In [ ]:
# **1**: How many orders were placed in total?
cursor.execute("SELECT COUNT(*) FROM orders")
total_orders = cursor.fetchone()
print("Total orders:", total_orders[0])

In [ ]:
# **2**: What is the total sales amount?
cursor.execute("SELECT SUM(total_sales) FROM orders")
total_sales = cursor.fetchone()
print("Total sales amount:", total_sales[0])

In [ ]:
# **3**: What is the average profit per order?
cursor.execute("SELECT AVG(profit) FROM orders")
average_profit = cursor.fetchone()
print("Average profit per order:", average_profit[0])

In [ ]:
# **4**: Which customer has placed the most orders?
cursor.execute("SELECT customer_id, COUNT(*) AS order_count FROM orders GROUP BY customer_id ORDER BY order_count DESC LIMIT 1")
top_customer = cursor.fetchone()
print("Customer with most orders:", top_customer)

In [ ]:
# **5**: What is the total profit for each product category?
cursor.execute("SELECT category, SUM(profit) FROM products GROUP BY category")
category_profits = cursor.fetchall()
for category, profit in category_profits:
    print(f"Category: {category}, Total Profit: {profit}")

In [ ]:
# **6**: What is the average discount percentage?
cursor.execute("SELECT AVG(discount_percentage) FROM orders")
avg_discount = cursor.fetchone()
print("Average discount percentage:", avg_discount[0])

In [ ]:
# **7**: Which region has the highest total sales?
cursor.execute("SELECT region, SUM(total_sales) FROM orders GROUP BY region ORDER BY SUM(total_sales) DESC LIMIT 1")
highest_sales_region = cursor.fetchone()
print("Region with highest total sales:", highest_sales_region)

In [ ]:
# **8**: How many orders were shipped using each shipping mode?
cursor.execute("SELECT shipping_mode, COUNT(*) FROM orders GROUP BY shipping_mode")
shipping_counts = cursor.fetchall()
for mode, count in shipping_counts:
    print(f"Shipping Mode: {mode}, Orders: {count}")

In [ ]:
# **9**: What is the total quantity ordered for each product sub-category?
cursor.execute("SELECT sub_category, SUM(quantity_ordered) FROM products GROUP BY sub_category")
sub_category_quantities = cursor.fetchall()
for sub_category, quantity in sub_category_quantities:
    print(f"Sub-category: {sub_category}, Total Quantity: {quantity}")

In [ ]:
# **10**: Which product has the highest profit margin?
cursor.execute("SELECT product_id, MAX(profit_margin) FROM products")
highest_margin_product = cursor.fetchone()
print("Product with highest profit margin:", highest_margin_product)

In [ ]:
# **11**: What is the total sales amount for each year?
cursor.execute("SELECT YEAR(order_date), SUM(total_sales) FROM orders GROUP BY YEAR(order_date)")
yearly_sales = cursor.fetchall()
for year, sales in yearly_sales:
    print(f"Year: {year}, Total Sales: {sales}")

In [ ]:
# **12**: Which customer segment has the highest average sales?
cursor.execute("SELECT customer_segment, AVG(total_sales) FROM orders GROUP BY customer_segment ORDER BY AVG(total_sales) DESC LIMIT 1")
highest_avg_sales_segment = cursor.fetchone()
print("Customer segment with highest average sales:", highest_avg_sales_segment)

# Close the connection
cursor.close()
db.close()

## PostgreSQL

**1**: What is the average age of the customers?

**2**: How many customers have defaulted on loans?

**3**: What is the maximum balance in the account?

**4**: How many customers have a housing loan?

**5**: What is the education level of the youngest customer?

**6**: What is the marital status of the oldest customer?

**7**: What is the distribution of the months in which customers were last contacted?

**8**: What is the average duration of contact for customers who have a personal loan?

**9**: How many customers were previously contacted, and what was the average duration of their last contact?

**10**: What is the distribution of job types among customers?

**11**: How many customers have been contacted more than once in the current campaign?

**12**: What is the success rate of previous marketing campaigns (poutcome) among customers